# Test multi-phenomenon append

Test append operation between four NetCDF datasets with a single append dimension, scalar coordinates and a different phenomenon name for each.

In [ ]:
import os
import tempfile

import nctotdb

In [ ]:
data_path = os.path.join(os.path.dirname(os.path.abspath(".")), "data")

In [ ]:
base_datasets = [os.path.join(data_path, "xy_ts0.nc"),
                 os.path.join(data_path, "xy_tws0.nc")]
append_datasets = [os.path.join(data_path, "xy_ts1.nc"),
                   os.path.join(data_path, "xy_tws1.nc")]

In [ ]:
ncdmg = nctotdb.data_model.NCDataModelGroup(base_datasets)

In [ ]:
fp = tempfile.TemporaryDirectory()
array_filepath = fp.name
array_filepath

In [ ]:
tiledb_name = "multi_phenom_append"
append_dim = "time"
data_array_name = "data"

writer = nctotdb.TileDBWriter(ncdmg,
                              array_filepath=array_filepath,
                              array_name=tiledb_name,
                              unlimited_dims=append_dim)
writer.create_domains()

## Test 1. Append

In [ ]:
writer.append([append_datasets], append_dim, data_array_name,
              baselines={append_dim: append_datasets[0]},
              group=True, verbose=True)

## Test 2. Load appended array as Iris cube

In [ ]:
reader = nctotdb.TileDBReader(tiledb_name,
                              array_filepath=array_filepath,
                              data_array_name=data_array_name)
result = reader.to_iris()
result

In [ ]:
expected_shape = (2, 4, 3)
for cube in result:
    assert cube.shape == expected_shape, f"Shapes do not match for cube {cube.name()!r}. Got {result.shape}, expected {expected_shape}."

In [ ]:
fp.cleanup()